In [15]:
from helper_functions import *
import sklearn
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel

In [16]:
collection = full_df('25022021_0010_red')


In [17]:
# views_forwards_by_week(collection)

In [18]:
collection.isna().value_counts()

message_id  message_text  message_date  message_views  message_forwards  message_hasMedia  message_web_description  message_web_url  heritage
False       False         False         False          False             False             True                     True             False       46112
                                                                                           False                    False            False       21277
            True          False         False          False             False             True                     True             False       18361
            False         False         False          False             False             True                     False            False        2155
                                        True           True              False             True                     True             False         342
dtype: int64

#### Now I'm dropping all rows, where values for text, web_description and web_url are NaN

In [19]:
collection_texts = collection.dropna(axis=0, how='all', subset=['message_text', 'message_web_description', 'message_web_url']) 


In [20]:
# collection['doc'] = [nlp(message_text) if type(message_text) == str else None for message_text in collection.message_text]
# for doc in nlp.pipe(collection_texts['message_text']):
# collection_texts['entities'] = [[ent.text for ent in doc.ents] for doc in nlp.pipe(collection_texts['message_text'])]
token = []
for doc in nlp.pipe(collection_texts.message_text, disable=["tagger", "parser"]):
    # Do something with the doc here
    token.append([ent.text for ent in doc.ents])

In [21]:
collection_texts['tokens'] = token

In [22]:
collection_texts.heritage.value_counts()

ATTILAHILDMANN_24022021_222224.csv                  31442
Xavier_Naidoo_24022021_223823.csv                   11911
FreieMedienTV_24022021_223428.csv                    6545
verbindediepunkte_24022021_223622.csv                4456
Haintz_24022021_223255.csv                           4273
MICHAELWENDLEROFFICIAL_24022021_223739.csv           3306
Dr_Heinrich_Fiechtner_24022021_223230.csv            1889
tonitano_24022021_223352.csv                         1214
naomiseibt_24022021_222209.csv                       1198
ATTILAHILDMANN_2_24022021_223416.csv                  887
KoronaAlarm_24022021_223725.csv                       807
Anonymous_Germany_24022021_223408.csv                 690
Corona_ist_nicht_das_Problem_24022021_223221.csv      648
militaernews_24022021_223551.csv                      620
Name: heritage, dtype: int64

#### I want an equal distribution of rows for every category in heritage, so i need to slim the dataframe down to 600 entries each

In [56]:
to_append = []
i = 0
amount = 400
for heritage in collection_texts.heritage.unique():
    if i == 0:
        collection_slim = (collection_texts[collection_texts['heritage'] == heritage].nlargest(amount, 'message_id'))
        i += 1
    else:
        collection_slim = collection_slim.append(collection_texts[collection_texts['heritage'] == heritage].nlargest(amount, 'message_id'))
    

collection_slim.reset_index(drop=True, inplace=True)
collection_slim


,message_id,message_text,message_date,message_views,message_forwards,message_hasMedia,message_web_description,message_web_url,heritage,tokens
0,3889,Der unwiderlegbare Beweis kommt bald - wir sin...,2021-02-24 20:47:55+00:00,8638.0,38.0,True,The United States Supreme Court refused to rev...,https://ishort.ink/1f6y,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Sidney Powell, Obersten Gerichtshofs, Wahlbet..."
1,3887,💥💥💥\nhttps://www.kopp-verlag.de/a/die-plandemi...,2021-02-24 20:33:36+00:00,12660.0,1.0,True,"Von der Wahl des Virus über die Art, wie es ge...",https://www.kopp-verlag.de/a/die-plandemie?&6=...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,[]
2,3885,Gestern fand ein Senatsausschuss zum Kapitol-E...,2021-02-24 20:25:36+00:00,9675.0,31.0,True,"🚨🚨🚨 Liebe Zuschauer, wir brauchen Ihre Hilfe! ...",https://www.youtube.com/watch?v=SNEhe_GwA-8&fe...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Kapitol-Einbruch, Befragt, Kapitol-Polizisten..."
3,3884,Dunkelziffer Nebenwirkung Tod nach Corona-Impf...,2021-02-24 19:37:37+00:00,15894.0,160.0,True,Die Psychologin Ines Cremer aus Uhldingen beri...,https://rumble.com/ve423t-es-ist-zu-befrchten-...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Dunkelziffer, Corona-Impfung, ‼️ \n\n, Impfun..."
4,3883,X22 Report vom 23.2.2021 - Keine solche Agentu...,2021-02-24 15:15:17+00:00,28391.0,201.0,True,NaN,NaN,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[X22, Die Bühne]"
...,...,...,...,...,...,...,...,...,...,...
5595,5086,X22 Report vom 13.1.2021 - Schachmatt - [Tag N...,2021-01-15 00:02:36+00:00,56376.0,781.0,True,NaN,NaN,verbindediepunkte_24022021_223622.csv,"[X22, Tag Null]"
5596,5085,DRINGEND ARGENTINIER!! \n\nSIE SAGEN ES IM FER...,2021-01-15 00:00:40+00:00,56878.0,4146.0,True,NaN,NaN,verbindediepunkte_24022021_223622.csv,"[DRINGEND, ARGENTINIER!! \n\n, SAGEN ES, FERNS..."
5597,5084,Lou Dobbs kompletter Beitrag:\n\nhttps://www.y...,2021-01-14 23:24:29+00:00,98458.0,437.0,True,NaN,https://www.youtube.com/watch?v=xW9e9dUuTTE&fe...,verbindediepunkte_24022021_223622.csv,[Lou Dobbs]
5598,5083,Wird wohl grad eng für die Vögel!,2021-01-14 23:22:11+00:00,79616.0,70.0,False,NaN,NaN,verbindediepunkte_24022021_223622.csv,[]


,index,message_id,message_text,message_date,message_views,message_forwards,message_hasMedia,message_web_description,message_web_url,heritage,tokens
0,0,3889,Der unwiderlegbare Beweis kommt bald - wir sin...,2021-02-24 20:47:55+00:00,8638.0,38.0,True,The United States Supreme Court refused to rev...,https://ishort.ink/1f6y,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Sidney Powell, Obersten Gerichtshofs, Wahlbet..."
1,2,3887,💥💥💥\nhttps://www.kopp-verlag.de/a/die-plandemi...,2021-02-24 20:33:36+00:00,12660.0,1.0,True,"Von der Wahl des Virus über die Art, wie es ge...",https://www.kopp-verlag.de/a/die-plandemie?&6=...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,[]
2,4,3885,Gestern fand ein Senatsausschuss zum Kapitol-E...,2021-02-24 20:25:36+00:00,9675.0,31.0,True,"🚨🚨🚨 Liebe Zuschauer, wir brauchen Ihre Hilfe! ...",https://www.youtube.com/watch?v=SNEhe_GwA-8&fe...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Kapitol-Einbruch, Befragt, Kapitol-Polizisten..."
3,5,3884,Dunkelziffer Nebenwirkung Tod nach Corona-Impf...,2021-02-24 19:37:37+00:00,15894.0,160.0,True,Die Psychologin Ines Cremer aus Uhldingen beri...,https://rumble.com/ve423t-es-ist-zu-befrchten-...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[Dunkelziffer, Corona-Impfung, ‼️ \n\n, Impfun..."
4,6,3883,X22 Report vom 23.2.2021 - Keine solche Agentu...,2021-02-24 15:15:17+00:00,28391.0,201.0,True,NaN,NaN,MICHAELWENDLEROFFICIAL_24022021_223739.csv,"[X22, Die Bühne]"
...,...,...,...,...,...,...,...,...,...,...,...
8395,83481,4846,oh...in der Nacht klang das noch anders:,2020-12-28 06:55:08+00:00,56555.0,12.0,False,NaN,NaN,verbindediepunkte_24022021_223622.csv,[]
8396,83482,4845,Da ist gestern wohl was Grösseres passiert...h...,2020-12-28 06:53:34+00:00,60533.0,76.0,True,"WEST PALM BEACH, Fla. (AP) — President Donald ...",https://apnews.com/article/donald-trump-politi...,verbindediepunkte_24022021_223622.csv,[Grösseres passiert...https://apnews.com/artic...
8397,83483,4844,https://www.youtube.com/watch?v=wgoJkEOa9_E,2020-12-27 20:06:12+00:00,148483.0,742.0,True,Wir blicken weiter auf die historischen Ereign...,https://www.youtube.com/watch?v=wgoJkEOa9_E,verbindediepunkte_24022021_223622.csv,[]
8398,83484,4843,Heute 21:00 Livestream!,2020-12-27 17:36:11+00:00,78133.0,333.0,False,NaN,NaN,verbindediepunkte_24022021_223622.csv,[Livestream!]


In [57]:
collection_slim.sample(30)

,message_id,message_text,message_date,message_views,message_forwards,message_hasMedia,message_web_description,message_web_url,heritage,tokens
4738,1086,“Das Virus sei ein Werkzeug von Jüdinnen*Juden...,2021-01-16 21:02:46+00:00,5916.0,2.0,True,"Bill Gates, der uns zwangsimpfen will, Eliten,...",https://ze.tt/warum-verschwoerungserzaehlungen...,ATTILAHILDMANN_2_24022021_223416.csv,"[Virus, Jüdinnen*Juden, Israel, Virus, Virus, ..."
5570,5127,https://twitter.com/hiiipow3r2/status/13497537...,2021-01-16 16:26:04+00:00,88887.0,132.0,True,DC looks like they’re preparing to overturn th...,https://twitter.com/hiiipow3r2/status/13497537...,verbindediepunkte_24022021_223622.csv,[]
414,1446,❌AB MORGEN 08:30 UHR JEDEN TAG VOR DER STAATSA...,2021-02-22 01:53:08+00:00,382.0,2.0,True,NaN,NaN,KoronaAlarm_24022021_223725.csv,"[❌AB MORGEN, JEDEN, TAG, DEMONSTRIEREN‼\n\n, H..."
4306,7083,🔥ES REICHT! KOSMETIKERIN PACKT AUS!!\n\nEine w...,2021-02-07 19:37:37+00:00,24694.0,316.0,True,NaN,NaN,FreieMedienTV_24022021_223428.csv,"[ES REICHT!, KOSMETIKERIN, PACKT, AUS!!, Video..."
1318,5079,https://t.me/RA_Ludwig/1144\n\nMeine Stellungn...,2021-02-19 22:58:15+00:00,43254.0,162.0,True,Bereits am 26.Januar 2021 war dem Bundesgesund...,https://t.me/RA_Ludwig/1144,Haintz_24022021_223255.csv,"[https://t.me/RA_Ludwig/1144\n\nMeine, KVR, Mü..."
3050,1927,https://pflegefueraufklaerung.de/?p=602\n\n@Dr...,2021-02-13 10:00:25+00:00,21095.0,51.0,True,"Impfen, Corona, Zahlen usw",https://pflegefueraufklaerung.de/?p=602,Dr_Heinrich_Fiechtner_24022021_223230.csv,[]
3376,559,🇺🇸 Washington D.C\n\nSchwere Ausschreitungen z...,2021-01-06 18:39:21+00:00,7991.0,66.0,True,NaN,NaN,militaernews_24022021_223551.csv,"[🇸 Washington, Polizei, Demonstrierenden, Madi..."
3923,1058,https://twitter.com/mattwalshblog/status/13470...,2021-01-07 02:22:21+00:00,12997.0,14.0,True,NaN,NaN,naomiseibt_24022021_222209.csv,[]
706,894,🦊 DR. ANGELA MERKEL & IHRE SCHERGEN HABEN IN D...,2020-12-13 22:19:03+00:00,2327.0,17.0,True,NaN,NaN,KoronaAlarm_24022021_223725.csv,"[🦊 DR, ANGELA MERKEL & IHRE SCHERGEN, LETZTEN,..."
309,3515,https://www.tichyseinblick.de/daili-es-sential...,2021-02-12 21:35:45+00:00,68929.0,138.0,True,"Corona soll sich exponentiell verbreiten, bis ...",https://www.tichyseinblick.de/daili-es-sential...,MICHAELWENDLEROFFICIAL_24022021_223739.csv,[https://www.tichyseinblick.de/daili-es-sentia...


In [58]:
counter = Counter()
for row in collection_slim.tokens:
    for t in row:
        counter[t] += 1
print(counter.most_common(30))
print(f"Anzahl Tokens: {len(counter)}")

[('👉', 303), ('Deutschland', 257), ('Luftangriffe', 247), ('Telegram', 242), ('t.me/FreieMedienTV', 216), ('USA', 166), ('SchlauenFuchs', 140), ('❤', 135), ('Trump', 132), ('OneLove ❤', 119), ('💀🕸', 112), ('🕷🕸💀', 112), ('Folg', 112), ('🟥\n\n', 111), ('Corona', 110), ('durchzusetzen‼️\n\n', 102), ('Berlin', 96), ('NWO-Agenda‼️\n\n🟥', 96), ('DER', 94), ('Türkei', 78), ('🔸🔸', 77), ('AUF', 71), ('Armenien', 71), ('Israel', 70), ('Facebook', 70), ('🇸', 67), ('Zeit', 66), ('YouTube', 66), ('Luftangriff', 65), ('Twitter', 64)]
Anzahl Tokens: 13770


In [59]:
channel = collection_slim


In [60]:
def my_tokenize(value):
    if type(value) != str:
        return []
    tokens = value.split(" ")
    result = []
    for t in tokens:
        # Whitespace links und rechts weg
        t = t.strip()
        # Alles klein schreiben, Groß/Kleinschreibung soll keine Rolle spielen
        t = t.lower()
        # Min. 2 Buchstaben
        if len(t)>1:
            result.append(t)
    # Oder: result = [t.strip() for t in tokens]
    return result


feature_list = []
for index, post in channel.iterrows():
    feature_row_dict = {
        "heritage": post["heritage"]
    }
    tokens = [t for t in post.tokens]
    for token in counter:
        feature_row_dict[token] = token in tokens
    feature_list.append(feature_row_dict)
feature_df = pd.DataFrame(feature_list)
feature_df

,heritage,Sidney Powell,Obersten Gerichtshofs,Wahlbetrug,Pennsylvania,Gorsuch,Thomas,Trump,Kavanaugh,Amy Coney Barrett,...,IHRE VERPFLICHTUNG ERFÜLLT,REIHE,UNSERE....,ERFÜLLEN,NACHRICHTT,VERBREITEN!!!!,ARGENTINIEN,AUF!!! \n\nDoctors for Truth,Lou Dobbs,Honest
0,MICHAELWENDLEROFFICIAL_24022021_223739.csv,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,MICHAELWENDLEROFFICIAL_24022021_223739.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,MICHAELWENDLEROFFICIAL_24022021_223739.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,MICHAELWENDLEROFFICIAL_24022021_223739.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,MICHAELWENDLEROFFICIAL_24022021_223739.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,verbindediepunkte_24022021_223622.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5596,verbindediepunkte_24022021_223622.csv,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
5597,verbindediepunkte_24022021_223622.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5598,verbindediepunkte_24022021_223622.csv,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


X = feature_df.drop('heritage', axis=1)
y = feature_df['heritage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 42)

#classifier = DecisionTreeClassifier()
#classifier = ComplementNB()
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred, output_dict = True, zero_division = 0)['weighted avg'])

{'precision': 0.5377415038678152, 'recall': 0.4375, 'f1-score': 0.4522846577049858, 'support': 560}
